In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph,START, END
from langchain_core.messages import SystemMessage, HumanMessage
from typing import TypedDict, Literal, Annotated
from pydantic import BaseModel, Field
import operator
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
generator_llm = ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite')
optimizer_llm = ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite')
evaluator_llm = ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite')


In [4]:

class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="feedback for the tweet.")

In [5]:
structured_evaluator_llm  = evaluator_llm.with_structured_output(TweetEvaluation)



In [6]:
# state
class TweetState(TypedDict):

    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int

    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

In [7]:

def generate_tweet(state: TweetState):

    # prompt
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
        Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

        Rules:
        - Do NOT use question-answer format.
        - Max 280 characters.
        - Use observational humor, irony, sarcasm, or cultural references.
        - Think in meme logic, punchlines, or relatable takes.
        - Use simple, day to day english
        """)
        ]

    # send generator_llm
    response = generator_llm.invoke(messages).content

    # return response
    return {'tweet': response, 'tweet_history': [response]}

In [8]:
def evaluate_tweet(state: TweetState):

    # prompt
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses 
""")
]

    response = structured_evaluator_llm.invoke(messages)

    return {'evaluation':response.evaluation, 'feedback': response.feedback, 'feedback_history': [response.feedback]}

In [9]:

def optimize_tweet(state: TweetState):

    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]

    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1

    return {'tweet': response, 'iteration': iteration, 'tweet_history': [response]}

In [10]:
def route_evaluation(state: TweetState):

    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'

In [11]:

graph = StateGraph(TweetState)

graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')

graph.add_conditional_edges('evaluate', route_evaluation, {'approved': END, 'needs_improvement': 'optimize'})
graph.add_edge('optimize', 'evaluate')

workflow = graph.compile()

In [18]:
initial_state = {
    "topic": "Nepal politican ",
    "iteration": 1,
    "max_iteration": 5
}
result = workflow.invoke(initial_state)

In [19]:
result

{'topic': 'Nepal politican ',
 'tweet': 'Just saw a Nepalese politician giving a speech. Pretty sure he accidentally nominated himself for "Most Dramatic Pause" at the next international conference. The mountain goats were more excited. 🐐😂 #NepalPolitics #SpeechGoals',
 'evaluation': 'approved',
 'feedback': "The tweet is original, humorous, and uses a relatable comparison to mountain goats to emphasize the dramatic pause. Its length and use of hashtags make it suitable for Twitter's format, and it has good potential for virality due to its lighthearted critique and humor. The tweet successfully avoids common pitfalls like question-answer formats or setup-punchline structures.",
 'iteration': 1,
 'max_iteration': 5,
 'tweet_history': ['Just saw a Nepalese politician giving a speech. Pretty sure he accidentally nominated himself for "Most Dramatic Pause" at the next international conference. The mountain goats were more excited. 🐐😂 #NepalPolitics #SpeechGoals'],
 'feedback_history': ["T

In [15]:
for tweet in result['tweet_history']:
    print(tweet)

My brain trying to pronounce "srhberhb" vs. my brain trying to remember where I left my keys. One is a cosmic mystery, the other is apparently a lost civilization. #srhberhb #brainfog #sendhelp
